In [1]:
# Plot some vertical sections and maps from the WRF output of the CNTRL run (member 1) of the aerosol set of exp.

In [2]:
# Import relevant modules
import xarray as xr
import numpy as np
import pydap
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cfeature           # import features
import uuid
from scipy import interpolate
import scipy.io
from glob import glob
import os
from matplotlib.offsetbox import AnchoredText

In [35]:
crs = ccrs.PlateCarree()
def plot_background(ax):
    #ax.set_extent(extent_param)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5)
    #ax.add_feature(cfeature.STATES, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'fontsize': 14}
    gl.ylabel_style = {'fontsize': 14}
    return ax

def nan_gaussian_filter(field,sigma):
    """
    Function to smooth the field ignoring the NaNs.
    I follow the first answer here 
    https://stackoverflow.com/questions/18697532/gaussian-filtering-a-image-with-nan-in-python
    By default, the filter is truncated at 4 sigmas.
    """
    from scipy.ndimage import gaussian_filter
    
    field = np.double(field)
    
    # Take the original field and replace the NaNs with zeros.
    field0 = field.copy()
    field0[np.isnan(field)] = 0
    ff = gaussian_filter(field0, sigma=sigma)
    
    # Create the smoothed weight field.
    weight = 0*field.copy()+1
    weight[np.isnan(field)] = 0
    ww = gaussian_filter(weight, sigma=sigma)
    
    zz = ff/(ww*weight) # This rescale for the actual weights used in the filter and set to NaN where the field
                        # was originally NaN.
    return zz
  

In [15]:
# Set the relevant paths.

path2WRF = '/home/agostino/SST-impact_data/wrf/'

In [37]:
# Sigma to smooth the fields
sigma = 10 # 30 km

# Loop on time
str_start = '2020-02-02T00:00:00'
str_end = '2020-02-04T00:00:00'
instant = np.datetime64(str_start)

tt=0
while instant < np.datetime64(str_end):
    wrf_filename = 'wrfout_d02_' + str(instant)[0:10] +'_'+ str(instant)[11:]
    print(wrf_filename)
    
    ds = xr.open_dataset(path2WRF+wrf_filename)

    lon = ds.XLONG[0].values
    lat = ds.XLAT[0].values
    extent = [np.min(lon),np.max(lon),np.min(lat),np.max(lat)]

    sst = ds.SST[0].values
    smooth_sst = nan_gaussian_filter(sst,sigma)
    seamask = sst/sst

    u10 = ds.U10[0].values
    v10 = ds.V10[0].values

    wspd = np.sqrt(u10**2+v10**2)

    # Plot wind speed and SST.
    from matplotlib import cm
    plasma20 = cm.get_cmap('plasma', 20)
    viridis10 = cm.get_cmap('viridis', 10)

    fig = plt.figure(figsize=(14,6),facecolor='white')

    ax0 = fig.add_subplot(1, 2, 1, projection=crs)
    plot_background(ax0)
    pc0 = ax0.pcolormesh(lon,lat,wspd,vmin=0,vmax=10,shading='nearest',
                       transform=ccrs.PlateCarree(),cmap=plasma20)
    step = 50
    ax0.barbs(lon[::step,::step],lat[::step,::step],u10[::step,::step],v10[::step,::step],length=6)
    cbar0 = plt.colorbar(pc0)
    cbar0.ax.tick_params(labelsize=12) 
    cbar0.set_label('Wind speed [m/s]',size=14)
    ax0.set_extent(extent)

    ax1 = fig.add_subplot(1, 2, 2, projection=crs)
    plot_background(ax1)
    ax1.contour(lon,lat,(smooth_sst-273.15)*seamask,levels=np.arange(0,30),colors='w',transform=ccrs.PlateCarree())
    pc1 = ax1.pcolormesh(lon,lat,(sst-273.15)*seamask,vmin=25,vmax=30,shading='nearest',
                         transform=ccrs.PlateCarree(),cmap=viridis10)
    cbar1 = plt.colorbar(pc1)
    cbar1.ax.tick_params(labelsize=12) 
    cbar1.set_label('SST [°C]',size=14)
    ax1.set_extent(extent)

    plt.suptitle(wrf_filename,fontsize=16)
    figure_name = 'wind_sst_map_' + "{:02}".format(tt) + '.png'
    plt.savefig('./maps_wrf/'+figure_name,bbox_inches='tight')

    plt.close(fig)
    
    tt += 1
    instant += np.timedelta64(1,'h')

wrfout_d02_2020-02-02_00:00:00


<ipython-input-37-d7ff0daf3ff0>:20: RuntimeWarning: invalid value encountered in true_divide
  seamask = sst/sst


wrfout_d02_2020-02-02_01:00:00
wrfout_d02_2020-02-02_02:00:00
wrfout_d02_2020-02-02_03:00:00
wrfout_d02_2020-02-02_04:00:00
wrfout_d02_2020-02-02_05:00:00
wrfout_d02_2020-02-02_06:00:00
wrfout_d02_2020-02-02_07:00:00
wrfout_d02_2020-02-02_08:00:00
wrfout_d02_2020-02-02_09:00:00
wrfout_d02_2020-02-02_10:00:00
wrfout_d02_2020-02-02_11:00:00
wrfout_d02_2020-02-02_12:00:00
wrfout_d02_2020-02-02_13:00:00
wrfout_d02_2020-02-02_14:00:00
wrfout_d02_2020-02-02_15:00:00
wrfout_d02_2020-02-02_16:00:00
wrfout_d02_2020-02-02_17:00:00
wrfout_d02_2020-02-02_18:00:00
wrfout_d02_2020-02-02_19:00:00
wrfout_d02_2020-02-02_20:00:00
wrfout_d02_2020-02-02_21:00:00
wrfout_d02_2020-02-02_22:00:00
wrfout_d02_2020-02-02_23:00:00
wrfout_d02_2020-02-03_00:00:00
wrfout_d02_2020-02-03_01:00:00
wrfout_d02_2020-02-03_02:00:00
wrfout_d02_2020-02-03_03:00:00
wrfout_d02_2020-02-03_04:00:00
wrfout_d02_2020-02-03_05:00:00
wrfout_d02_2020-02-03_06:00:00
wrfout_d02_2020-02-03_07:00:00
wrfout_d02_2020-02-03_08:00:00
wrfout_d